In [3]:
pip install customtkinter


Note: you may need to restart the kernel to use updated packages.


In [1]:
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import threading

In [2]:
model = load_model("pcos_detection_model.h5")

2025-01-25 16:55:20.803861: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-01-25 16:55:20.803907: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-01-25 16:55:20.803917: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-01-25 16:55:20.804003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-25 16:55:20.804299: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def preprocess_single_image(img_path, img_size=(128, 128)):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.resize(img, img_size) 
    img = img / 255.0  
    img = np.expand_dims(img, axis=0)  
    return img

def predict_image(img_path):
    test_img = preprocess_single_image(img_path)
    if test_img is None:
        return None
    prediction = model.predict(test_img)
    class_label = np.argmax(prediction, axis=1)
    return "Infected" if class_label == 0 else "Non-Infected"

def run_prediction(img_path):
    def thread_task():
        # Display processing message
        spinner_label.configure(text="⏳ Processing...")
        result_label.configure(text="", fg_color="#2E2E2E")
        
        result = predict_image(img_path)
        
        spinner_label.configure(text="")
        if result:
            result_label.configure(
                text=f"Prediction: {result}",
                fg_color="#32CD32" if result == "Non-Infected" else "#FF4500",
            )
        else:
            result_label.configure(text="Error: Could not load image.", fg_color="#FF0000")

    threading.Thread(target=thread_task).start()

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img = img.resize((300, 300))  
        img = ImageTk.PhotoImage(img)
        image_label.configure(image=img, text="") 
        image_label.image = img
        run_prediction(file_path)

ctk.set_appearance_mode("Dark") 
ctk.set_default_color_theme("green")
root = ctk.CTk()
root.title("PCOS Detection System")
root.geometry("700x700")

title_label = ctk.CTkLabel(
    root,
    text="PCOS Detection System",
    font=("Arial Bold", 28),
    fg_color="#FFFFFF",
    corner_radius=10,
    text_color="black",
    padx=20,
    pady=10,
)
title_label.pack(pady=20)

image_frame = ctk.CTkFrame(root, width=320, height=320, corner_radius=15)
image_frame.pack(pady=10)
image_label = ctk.CTkLabel(image_frame, text="Upload an Image", font=("Arial", 16))
image_label.pack(expand=True)

upload_button = ctk.CTkButton(
    root,
    text="Upload Image",
    command=upload_image,
    font=("Arial", 18),
    fg_color="#4CAF50",
    hover_color="#45A049",
    corner_radius=12,
    width=200,
)
upload_button.pack(pady=20)

spinner_label = ctk.CTkLabel(root, text="", font=("Arial", 18), text_color="#FFA500")
spinner_label.pack(pady=10)

result_label = ctk.CTkLabel(
    root,
    text="",
    font=("Arial", 18),
    fg_color="#2E2E2E",
    text_color="white",
    corner_radius=10,
    width=350,
    height=50,
    anchor="center",
    padx=20,
    pady=10,
)
result_label.pack(pady=30)

footer_label = ctk.CTkLabel(
    root,
    text="Developed by UDITHA",
    font=("Arial Italic", 12),
    text_color="gray",
)
footer_label.pack(side="bottom", pady=10)

root.mainloop()

2025-01-25 16:58:23.783 python[26824:20863026] +[IMKClient subclass]: chose IMKClient_Legacy
2025-01-25 16:58:23.783 python[26824:20863026] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
2025-01-25 16:58:25.430 python[26824:20863026] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
/Users/rs_en/tensorflow-test/env/lib/python3.8/site-packages/customtkinter/windows/widgets/core_widget_classes/ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


1/1 [==============================] - 0s 320ms/step


2025-01-25 16:58:33.342188: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 15ms/step


libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
